In [32]:
import json
from pathlib import Path
import numpy as np
import imageio
import pandas as pd
from bqplot import (
    Figure,
    Scatter,
    LinearScale,
    Axis,
    Toolbar,
    Label,
    ColorAxis,
    ColorScale,
    OrdinalColorScale
)
import ipywidgets as widgets


import base64

In [45]:

def html_thumb(thumb):
    return f"""<img src="data:image/jpg;base64,{thumb}">"""

class MeasureViz(widgets.VBox):
    
    def __init__(self, measure_hdf, thumbs_json):
        
        self.measures = pd.read_hdf(measure_hdf)
        with open(thumbs_json, 'r') as fh:
            thumbs = json.load(fh)
        self.thumbs = {int(k): th for k, th in thumbs.items()}
        self.goto = widgets.HTML('Open in DB')
        x_sc = LinearScale()
        y_sc = LinearScale()
        c_sc = ColorScale(scheme="PuBuGn")
    
        c_ord = OrdinalColorScale(colors=['DodgerBlue', 'SeaGreen', 'HotPink', 'OrangeRed'])
        self.scat = Scatter(
            x=self.measures["SNR"],
            y=self.measures["resolution"].clip(0, 20),
            color=self.measures["C"],
            scales={
                "x": x_sc,
                "y": y_sc,
                "color": c_ord,
            },
            names=self.measures["Id"],
            display_names=False,
            fill=True,
            default_opacities=[0.6,],
        )
        ax_x = Axis(
            scale=x_sc,
            label="SNR",
            tick_format='0.2f'
        )
        ax_y = Axis(
            scale=y_sc,
            label='Resolution',
            orientation="vertical"
        )
    
        self.scat.on_element_click(self.goto_db)
        self.scat.on_hover(self.show_thumb)
        self.scat.tooltip = widgets.HTML("")

        self.fig = Figure(
            marks=[self.scat,],
            axes=[ax_x, ax_y],
        )
        super().__init__([self.fig, self.goto])

    def show_thumb(self, cbk, target):
        name = target['data']["name"] 
        self.scat.tooltip = widgets.HTML(html_thumb(self.thumbs.get(name, "")))

    def goto_db(self, cbk, target):
        name = target['data']["name"]
        html_ = html_thumb(self.thumbs.get(name, ""))
        if html_:
            url = f"http://139.124.81.38:4080/webclient/img_detail/{name}"
            self.goto.value = (f'<a href={url} target="_blank">{html_}</a>')


In [47]:
base = Path("/home/guillaume/AmuBox/DataExchange/")
measure_hdf = base / "measures_10452_2020-04-20.hf5"
thumbs = base / "thumbs.json"

mv = MeasureViz(measure_hdf, thumbs)
mv

MeasureViz(children=(Figure(axes=[Axis(label='SNR', scale=LinearScale(), tick_format='0.2f'), Axis(label='Reso…

In [50]:
mv.measures[mv.measures["SNR"]==0]

,Id,AquisitionDate,LensNA,ChannelLabel,PhysicalSizeX,C,Z,T,SNR,resolution
0,16437,2015-04-09 02:09:49,0.8,Ch2-T1,0.415133,0.0,0.0,0.0,0.0,2.0
1,16437,2015-04-09 02:09:49,0.8,Ch1-T2,0.415133,1.0,0.0,0.0,0.0,2.0
2,16437,2015-04-09 02:09:49,0.8,Ch2-T3,0.415133,2.0,0.0,0.0,0.0,2.0
1,16407,2015-04-09 02:09:49,0.8,Ch1-T2,0.415133,1.0,0.0,0.0,0.0,2.0
1,16404,2015-04-09 02:09:49,0.8,Ch1-T2,0.415133,1.0,0.0,0.0,0.0,2.0
2,16404,2015-04-09 02:09:49,0.8,Ch2-T3,0.415133,2.0,0.0,0.0,0.0,2.0
1,16409,2015-04-09 02:09:49,0.8,Ch1-T2,0.415133,1.0,0.0,0.0,0.0,2.0
1,16406,2015-04-09 02:09:49,0.8,Ch1-T2,0.415133,1.0,0.0,0.0,0.0,2.0
0,16438,2015-04-09 02:09:49,0.8,Ch2-T1,0.415133,0.0,0.0,0.0,0.0,2.0
1,16438,2015-04-09 02:09:49,0.8,Ch1-T2,0.415133,1.0,0.0,0.0,0.0,2.0
